# L=2 BMN SU2 matrix model

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
import qiskit
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_algorithms import NumPyEigensolver
import time
import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

# Load H

In [2]:
def open_n_process(file_link):
    file = open(file_link, "r")
    content = file.read()

    data = content.split(',\n')
    data[0] = data[0].split('\n')[1]
    data[-1] = data[-1].split('\n')[0]

    data_coeff = []
    data_str = []
    for i in range(len(data)):
        data_coeff.append(float(data[i].split('*')[0]))
        data_str.append(data[i].split('*')[1].split(' ')[1])
    return data_coeff, data_str
    
def check_min_max(data_coeff):
    print(f'Min absolute value is {np.round(np.min(np.abs(data_coeff[1:])),6)}')
    print(f'Max absolute value is {np.round(np.max(np.abs(data_coeff[1:])),5)}')
    print(f'Mean absolute value is {np.round(np.mean(np.abs(data_coeff[1:])),5)}')
    
def get_k_largest_ops(data_coeff, data_str, k):
    ind_list = np.argsort(np.abs(data_coeff[1:]))[-k:]
    ops = [data_str[1:][i] for i in ind_list]
    val= [np.round(data_coeff[1:][i],4) for i in ind_list]
    return ind_list, ops, val

In [3]:
gg = '02'
data_coeff, data_str = open_n_process(f"../utility/L2_BMN_l{gg}_Hpauli.txt")

In [4]:
Hpauli = list(zip(data_str, data_coeff))
H4q = SparsePauliOp.from_list(Hpauli)
# exactly diagonalize the system using numpy routines
solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
#print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],6)
print(f'{check_min_max(data_coeff)}')
print(f'E_exact = {E_exact}')

Min absolute value is 0.05
Max absolute value is 0.75
Mean absolute value is 0.30406
None
E_exact = 0.003287


In [5]:
Hpauli

[('IIIIIIIII', 5.400000000000002),
 ('ZIIIIIIII', -0.5),
 ('IZIIIIIII', -0.5),
 ('IIZIIIIII', -0.5),
 ('IIIZIIIII', -0.5),
 ('IIIIZIIII', -0.5),
 ('IIIIIZIII', -0.5),
 ('IIIIIIZII', -0.75),
 ('IIIIIIIZI', -0.75),
 ('IIIIIIIIZ', -0.75),
 ('XXIXXIIII', -0.04999999999999999),
 ('XIXXIXIII', -0.04999999999999999),
 ('IXXIXXIII', -0.04999999999999999),
 ('IIXIIIYXI', 0.15811388300841894),
 ('IIXIIIXYI', 0.15811388300841894),
 ('IIIIIXXXI', 0.15811388300841894),
 ('IIIIIXYYI', -0.15811388300841894),
 ('IXIIIIYZX', -0.15811388300841894),
 ('IXIIIIXZY', -0.15811388300841894),
 ('IIIIXIXZX', -0.15811388300841894),
 ('IIIIXIYZY', 0.15811388300841894),
 ('XIIIIIIYX', 0.15811388300841894),
 ('XIIIIIIXY', 0.15811388300841894),
 ('IIIXIIIXX', 0.15811388300841894),
 ('IIIXIIIYY', -0.15811388300841894)]

# EvolvedOperatorAnsatz

In [5]:
ind15, ops15, vals15 = get_k_largest_ops(data_coeff, data_str, 15)
ind20, ops20, vals20 = get_k_largest_ops(data_coeff, data_str, 20)

ops_Hp15 =[Pauli(f'{ops15[i]}') for i in range(len(ops15))]
ops_Hp20 =[Pauli(f'{ops20[i]}') for i in range(len(ops20))]
ops_H =[Pauli(f'{data_str[1:][i]}') for i in range(len(data_str[1:]))]

In [6]:
ev_op_Hp15 = EvolvedOperatorAnsatz(ops_Hp15, reps=1, insert_barriers=True)
ev_op_Hp20 = EvolvedOperatorAnsatz(ops_Hp20, reps=1, insert_barriers=True)
ev_op_H = EvolvedOperatorAnsatz(ops_H, reps=1, insert_barriers=True)

ev_op_Hp15_2f = EvolvedOperatorAnsatz(ops_Hp15, reps=2, insert_barriers=True)
ev_op_Hp20_2f = EvolvedOperatorAnsatz(ops_Hp20, reps=2, insert_barriers=True)
ev_op_H_2f = EvolvedOperatorAnsatz(ops_H, reps=2, insert_barriers=True)

ev_op_Hp15_3f = EvolvedOperatorAnsatz(ops_Hp15, reps=3, insert_barriers=True)
ev_op_Hp20_3f = EvolvedOperatorAnsatz(ops_Hp20, reps=3, insert_barriers=True)
ev_op_H_3f = EvolvedOperatorAnsatz(ops_H, reps=3, insert_barriers=True)

ev_op_Hp15_4f = EvolvedOperatorAnsatz(ops_Hp15, reps=4, insert_barriers=True)
ev_op_Hp20_4f = EvolvedOperatorAnsatz(ops_Hp20, reps=4, insert_barriers=True)
ev_op_H_4f = EvolvedOperatorAnsatz(ops_H, reps=4, insert_barriers=True)

ansatz_list = [ev_op_Hp15, ev_op_Hp20, ev_op_H, 
               ev_op_Hp15_2f, ev_op_Hp20_2f, ev_op_H_2f,
               ev_op_Hp15_3f, ev_op_Hp20_3f, ev_op_H_3f,
               ev_op_Hp15_4f, ev_op_Hp20_4f, ev_op_H_4f]

ansatz_names = ['ev_op_Hp15', 'ev_op_Hp20', 'ev_op_H', 
               'ev_op_Hp15_2f','ev_op_Hp20_2f','ev_op_H_2f',
               'ev_op_Hp15_3f','ev_op_Hp20_3f','ev_op_H_3f',
               'ev_op_Hp15_4f','ev_op_Hp20_4f','ev_op_H_4f']

print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [15, 20, 24, 30, 40, 48, 45, 60, 72, 60, 80, 96]


In [8]:
print(*list(zip(ops20, vals20)), sep = '\n')

('XIIIIIIYX', 0.1581)
('IIIIXIYZY', 0.1581)
('IIIIXIXZX', -0.1581)
('IXIIIIXZY', -0.1581)
('IXIIIIYZX', -0.1581)
('IIIIIXYYI', -0.1581)
('IIIIIXXXI', 0.1581)
('IIXIIIXYI', 0.1581)
('IIIXIIIYY', -0.1581)
('IIIXIIIXX', 0.1581)
('IIXIIIYXI', 0.1581)
('IIIIIZIII', -0.5)
('IIIIZIIII', -0.5)
('IIIZIIIII', -0.5)
('IIZIIIIII', -0.5)
('IZIIIIIII', -0.5)
('ZIIIIIIII', -0.5)
('IIIIIIIIZ', -0.75)
('IIIIIIIZI', -0.75)
('IIIIIIZII', -0.75)


In [9]:
print(*list(zip(ops15, vals15)), sep = '\n')

('IIIIIXYYI', -0.1581)
('IIIIIXXXI', 0.1581)
('IIXIIIXYI', 0.1581)
('IIIXIIIYY', -0.1581)
('IIIXIIIXX', 0.1581)
('IIXIIIYXI', 0.1581)
('IIIIIZIII', -0.5)
('IIIIZIIII', -0.5)
('IIIZIIIII', -0.5)
('IIZIIIIII', -0.5)
('IZIIIIIII', -0.5)
('ZIIIIIIII', -0.5)
('IIIIIIIIZ', -0.75)
('IIIIIIIZI', -0.75)
('IIIIIIZII', -0.75)


# VQE

In [8]:
seed = 170
iterations = 2000
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(iterations, optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.6f}")
    return result

# COBYLA

At step 5, with ev_op_H_2f
VQE result: 0.030991
Length of this optimization 849, time taken = 177.049 

In [11]:
r_cobyla=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15
VQE result: 2.055090
Length of this optimization 192, time taken = 15.519 

At step 1, with ev_op_Hp20
VQE result: 0.078069
Length of this optimization 276, time taken = 36.332 

At step 2, with ev_op_H
VQE result: 2.147212
Length of this optimization 344, time taken = 60.604 

At step 3, with ev_op_Hp15_2f
VQE result: 0.092912
Length of this optimization 350, time taken = 50.628 

At step 4, with ev_op_Hp20_2f
VQE result: 0.072005
Length of this optimization 533, time taken = 126.417 

At step 5, with ev_op_H_2f
VQE result: 0.030991
Length of this optimization 849, time taken = 196.843 

At step 6, with ev_op_Hp15_3f
VQE result: 0.093582
Length of this optimization 516, time taken = 80.066 

At step 7, with ev_op_Hp20_3f
VQE result: 0.167037
Length of this optimization 1061, time taken = 284.068 

At step 8, with ev_op_H_3f
VQE result: 2.204857
Length of this optimization 1038, time taken = 344.156 

At step 9, with ev_op_Hp15_4f
VQE result: 0.100251
Length 

In [12]:
df1 = pd.concat([r_cobyla[i] for i in range(len(r_cobyla))], axis = 1)
df1.to_csv(f'L2_l{gg}_op_ev_cobyla.csv')

# SPSA

At step 3, with ev_op_Hp15_2f
VQE result: 0.071424
Length of this optimization 3551, time taken = 308.376 

At step 3, with ev_op_Hp15_2f
VQE result: 0.091715
Length of this optimization 4051, time taken = 360.693 

At step 1, with ev_op_Hp20
VQE result: 0.012277
Length of this optimization 4051, time taken = 413.364

In [13]:
r_spsa=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 0.093538
Length of this optimization 4051, time taken = 261.896 

At step 1, with ev_op_Hp20
VQE result: 0.012277
Length of this optimization 4051, time taken = 413.364 

At step 2, with ev_op_H
VQE result: 0.118274
Length of this optimization 4051, time taken = 462.968 

At step 3, with ev_op_Hp15_2f
VQE result: 0.091715
Length of this optimization 4051, time taken = 360.693 

At step 4, with ev_op_Hp20_2f
VQE result: 0.052515
Length of this optimization 4051, time taken = 595.173 

At step 5, with ev_op_H_2f
VQE result: 1.534141
Length of this optimization 4051, time taken = 703.427 

At step 6, with ev_op_Hp15_3f
VQE result: 0.097625
Length of this optimization 4051, time taken = 463.019 

At step 7, with ev_op_Hp20_3f
VQE result: 0.181843
Length of this optimization 4051, time taken = 774.16 

At step 8, with ev_op_H_3f
VQE result: 2.207661
Length of this optimization 4051, time taken = 916.238 

At step 9, with ev_op_Hp15_4f
VQE result: 0.135

In [14]:
df2 = pd.concat([r_spsa[i] for i in range(len(r_spsa))], axis = 1)
df2.to_csv(f'L2_l{gg}_op_ev_spsa.csv')

## Retrain the best performing ansatz

At step 1, with ev_op_Hp20
VQE result: 0.012277
Length of this optimization 4051, time taken = 413.364

In [11]:
#Increasing the number of iterations makes it worse
iterations = 2500
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 1.987036
Length of this optimization 5051, time taken = 598.669 



In [13]:
#Reducing the number of iterations
iterations = 1750
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 0.065094
Length of this optimization 3551, time taken = 472.251 



In [14]:
#Reducing the number of iterations
iterations = 1800
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 2.993583
Length of this optimization 3651, time taken = 509.371 



In [18]:
df = pd.read_csv('results/L2_l02_op_ev_spsa.csv')
dfp = df['ev_op_Hp20']
dfp.to_csv(f'results/L2_l{gg}_op_ev_spsa_best.csv')

# SLSQP

In [15]:
r_slsqp=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, SLSQP, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_slsqp.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 5.574897
Length of this optimization 16, time taken = 2.441 

At step 1, with ev_op_Hp20
VQE result: 5.013021
Length of this optimization 21, time taken = 3.657 

At step 2, with ev_op_H
VQE result: 5.817232
Length of this optimization 25, time taken = 4.448 

At step 3, with ev_op_Hp15_2f
VQE result: 4.469303
Length of this optimization 31, time taken = 3.903 

At step 4, with ev_op_Hp20_2f
VQE result: 5.274540
Length of this optimization 41, time taken = 9.728 

At step 5, with ev_op_H_2f
VQE result: 5.221282
Length of this optimization 49, time taken = 12.629 

At step 6, with ev_op_Hp15_3f
VQE result: 3.554969
Length of this optimization 46, time taken = 7.79 

At step 7, with ev_op_Hp20_3f
VQE result: 5.584767
Length of this optimization 61, time taken = 19.025 

At step 8, with ev_op_H_3f
VQE result: 5.208698
Length of this optimization 73, time taken = 25.83 

At step 9, with ev_op_Hp15_4f
VQE result: 4.232864
Length of this optimization 61

In [16]:
df3 = pd.concat([r_slsqp[i] for i in range(len(r_slsqp))], axis = 1)
df3.to_csv(f'L2_l{gg}_op_ev_slsqp.csv')